In [1]:
import pandas as pd
from pathlib import Path
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
import re
from bs4 import BeautifulSoup
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/venkateshmunaga/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/venkateshmunaga/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
csv_path = Path.cwd().parent / "data" / "index feature extract.csv"
df = pd.read_csv(csv_path)

In [3]:
df

,rec_id,url,website,result,created_date,tld,url_len,is_domain_IP,no_of_sub_domain,no_of_obfuscated_chars,...,has_title,has_description,has_external_form_submit,has_faviocn,no_of_images,no_of_js,has_password_field,has_copyright_info,has_hidden_field,no_financial_terms
0,1,http://intego3.info/EXEL/index.php,1613573972338075.html,1,2021-02-17 20:29:32,info,34,0,0,0,...,1,0,0,1,4,1,0,0,0,0
1,2,https://www.mathopenref.com/segment.html,1635698138155948.html,0,2021-10-31 16:35:38,com,40,0,1,0,...,1,0,0,0,2,4,0,0,1,0
2,3,https://www.computerhope.com/issues/ch000254.htm,1635699228889266.html,0,2021-10-31 16:53:48,com,48,0,1,0,...,1,1,0,1,5,8,0,1,0,1
3,4,https://www.investopedia.com/terms/n/next-elev...,1635750062162701.html,0,2021-11-01 12:31:02,com,52,0,1,0,...,1,1,0,1,19,6,0,0,0,3
4,5,https://jobs.emss.org.uk/lcc.aspx,161356510250721.html,0,2021-02-17 18:01:42,org.uk,33,0,1,0,...,1,1,0,1,12,17,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79995,79996,https://bestjobmanage.com/adminservicedesk/True/,1622125898152592.html,1,2021-05-27 14:31:38,com,48,0,0,0,...,1,0,0,1,3,2,0,1,1,0
79996,79997,http://vieuxshack.com/download/adobe/b51f18074...,160822961118687.html,1,2020-12-17 23:56:51,com,245,0,0,8,...,1,0,0,1,1,1,0,0,0,0
79997,79998,https://www.tumblr.com/search/gas%20mask%20tattoo,163570726902772.html,0,2021-10-31 19:07:49,com,49,0,1,4,...,1,1,0,1,38,8,0,0,0,1
79998,79999,https://www.magnetic-shield.com/pdf/wc_4.pdf,1635701992217159.html,0,2021-10-31 17:39:52,com,44,0,1,0,...,1,1,0,1,15,18,0,1,0,1


In [4]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
html_files = df.website.values

corpus_words_toknized = []

def corpus_generator(document: str) -> list[str]:
    corpus = sent_tokenize(document)
    corpus = [re.sub(r'[^a-zA-Z0-9\s]', '', sent) for sent in corpus]
    return corpus


for file in tqdm(html_files):
    html_file_path = Path.cwd().parent / "data" / "html data" / file
    html_file = html_file_path.read_text(encoding="utf-8", errors="ignore")
    soup = BeautifulSoup(html_file, 'lxml')
    document = soup.get_text(separator=" ", strip=True).lower()
    courps = corpus_generator(document=document)
    for sent in courps:
        words_tokens = word_tokenize(sent)
        corpus_words_toknized.append(" ".join([lemmatizer.lemmatize(word) for word in words_tokens if word not in stop_words]))

  0%|          | 0/80000 [00:00<?, ?it/s]

100%|██████████| 80000/80000 [14:10<00:00, 94.02it/s] 


In [5]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=400)
vectorizer.fit(corpus_words_toknized)

,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None
,analyzer,'word'
,stop_words,None
,token_pattern,'(?u)\\b\\w\\w+\\b'
,ngram_range,"(1, ...)"


In [6]:
file_path = Path.cwd().parent / "data" / "html data" / "1608573404300907.html"
file = file_path.read_text(encoding="utf-8", errors="ignore")
soup = BeautifulSoup(html_file, 'lxml')
document = soup.get_text(separator=" ", strip=True).lower()
doc_vec = vectorizer.transform([document])

In [7]:
filename = Path.cwd().parent / "models" / "tfidf_vectorizer.pkl"
with open(file=filename, mode='wb') as file:
    pickle.dump(obj=vectorizer, file=file)